CIFAR-10 Base Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import seaborn as sns


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [3]:
train_data = datasets.CIFAR10(root='./data', train=True, download=False, transform=transforms.ToTensor())
test_data=datasets.CIFAR10(root='./data', train=False, download=False, transform=transforms.ToTensor())

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=4, num_workers=2, pin_memory=True)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=4, num_workers=2, pin_memory=True)

images, labels=next(iter(train_dataloader))

In [4]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor=nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
        )
        self.classifier=nn.Linear(128 * 4 * 4, num_classes)
    def forward(self, x):
        x=self.feature_extractor(x)
        x=self.classifier(x)
        return(x)

In [5]:
net= Net(num_classes=10).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output=net(images)
        loss=criterion(output, labels)
        loss.backward()
        optimizer.step()
    

In [6]:
from torchmetrics import Recall, Precision, Accuracy
metric_precision= Precision(task='multiclass', num_classes=10, average='macro')
metric_recall=Recall(task='multiclass', num_classes=10, average='macro')
metric_acc=Accuracy(task='multiclass', num_classes=10, average='micro')

net.eval()
with torch.no_grad():
    for images, labels in tqdm(test_dataloader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        output = net(images)
        preds = torch.argmax(output, dim=1)
        metric_acc(preds.cpu(), labels.cpu())   
        metric_precision(preds.cpu(), labels.cpu())
        metric_recall(preds.cpu(), labels.cpu())


precision= metric_precision.compute()
recall= metric_recall.compute()
accuracy= metric_acc.compute()

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f" Accuracy: {accuracy}")

Evaluating: 100%|██████████| 2500/2500 [00:21<00:00, 115.52it/s]

Precision: 0.6865
Recall: 0.6669
 Accuracy: 0.6668999791145325


In [ ]:
print(test_data.class_to_idx)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


macro :
    Precision: 0.6626312732696533
    Recall: 0.6531000137329102

micro:
    Precision: tensor([0.7547, 0.8243, 0.4681, 0.4786, 0.6607, 0.5382, 0.6672, 0.7100, 0.7332,
        0.7912])
    Recall: tensor([0.5660, 0.7880, 0.6540, 0.4250, 0.5570, 0.5560, 0.7740, 0.6610, 0.8300,
        0.7200])
    Accuracy: tensor([0.5660, 0.7880, 0.6540, 0.4250, 0.5570, 0.5560, 0.7740, 0.6610, 0.8300,
        0.7200])
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}

In [ ]:
print(torch.cuda.is_available()) 

False
